# Trabalho Prático 1
## Grupo:
* Gabriel Alves Reis, 
* Gabriel Castelo Branco Rocha Alencar Pinto, 2020006523
* Samuel Brísio, 


In [ ]:
# %pip install pandas
# %pip install matplotlib

1- Envoltória Convexa (Gift Wrapping ??)
- Implementar primitiva de posição relativa de semiretas
- Sorted (default do python) com nossa primitiva
- Algoritmo da Envoltória (Gift Wrapping)
    - (conj. de pontos) -> (conj. de pontos)
- função para visualizar a envoltória

In [ ]:
# imports

In [ ]:
class Dot:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __lt__(self, b):
        A = (self.x)*(b.y)
        B = (self.y)*(b.x)
        return (A - B) > 0
    
    def __sub__(self, b):
        return Dot(self.x - b.x, self.y - b.y)
    
    def __add__(self, b):
        return Dot(self.x + b.x, self.y + b.y)

    def __repr__(self):
        return f'({self.x}, {self.y})'
    
    def __eq__(self, other):
        if(self.x == other.x and self.y == other.x):
            return True
        return False

In [ ]:
class Endpoint:
    def __init__(self, dot:Dot, segmentIndx,  endpointType):
        # endpoit type = 'left' | 'right'
        self.dot = dot
        self.segmentIndx = segmentIndx
        self.endpointType = endpointType
    
    def __repr__(self):
        return f'({self.dot}, {self.segmentIndx}, {self.endpointType})'

    def __lt__(self, b):
        if self.dot.x == b.dot.x:
            if self.endpointType == b.endpointType:
                return self.dot.y < b.dot.y
            else:
                return True if self.endpointType == 'left' else False
        else: 
            return self.dot.x < b.dot.x

    def __eq__(self, other):
        if(self.dot == other.dot and self.segmentIndx == other.segmentIndx and self.endpointType == other.endpointType):
            return True
        return False

In [ ]:
# Define a classe Seguimento
# O segmento terá um ponto a esquerda, a direita e um rotulo informando a qual poligono pertence

class Segment:
    def __init__(self, left:Dot, right:Dot, label):
        self.left = left
        self.right = right
        self.label = label

    def __repr__(self):
        return f'( {self.left} -> {self.right}, {self.label} )'

    def __eq__(self, other):
        if(self.left == other.left and self.right == other.right and self.label == other.label):
            return True
        return False

In [ ]:
def sortDotsByPolarAngle(dotsParam):
    dots = dotsParam.copy()
    anchor = dots[0]
    indx = 0
    anchor_indx = indx
    # find anchor
    for p in dots:
        if p.y < anchor.y :
            anchor = p
            anchor_indx = indx
        elif (p.y == anchor.y and p.x < anchor.x) :
            anchor = p  
            anchor_indx = indx
        indx += 1

    # normalize dots
    norm_dots = []
    dots.pop(anchor_indx)
    for dot in dots:
        norm_dots.append(dot - anchor)
    
    norm_dots.sort()

    sorted_dots = [anchor]
    for dot in norm_dots:
        sorted_dots.append(dot + anchor)

    return sorted_dots

In [ ]:
import random
def noise(x):
    return x + random.random() / 100

In [ ]:
import matplotlib.pyplot as plt
from random import randint 
import random

dot_list = []

random.seed(16)
for i in range(30):
    x = randint(0,200)
    y = noise(randint(0,200))
    a = Dot(x, y)
    dot_list.append(a)

x = []
y = []

for e in sortDotsByPolarAngle(dot_list):
    x.append(e.x)
    y.append(e.y)

fig, ax = plt.subplots()
ax.plot(x, y, 'r*')

for i, txt in enumerate(range(len(x))):
    ax.annotate(txt, (x[i], y[i]))
plt.ylabel('some numbers')
plt.show()


# Envoltória

In [ ]:
def isLeftTurn(a, b, c):
    B = b - a
    C = c - a

    term1 = (B.x)*(C.y)
    term2 = (B.y)*(C.x)
    return (term1 - term2) > 0

In [ ]:
def Graham(DotListParam):
    DotList = DotListParam.copy()
    stack = []
    stack.append(DotList[0])
    stack.append(DotList[1])
    stack.append(DotList[2])

    for i in range(3, len(DotList), 1):
        laster = len(stack) - 1

        while not isLeftTurn(stack[-2], stack[-1], DotList[i]):
            stack.pop()
            laster -= 1
        stack.append(DotList[i])

    return stack

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y, 'r*')

for i, txt in enumerate(range(len(x))):
    ax.annotate(txt, (x[i], y[i]))
plt.ylabel('some numbers')

x_envolt = []
y_envolt = []

sorted_list = sortDotsByPolarAngle(dot_list)

convexEnvelope = Graham(sorted_list)
for e in convexEnvelope:
    x_envolt.append(e.x)
    y_envolt.append(e.y)


ax.plot(x_envolt, y_envolt, 'b-')
ax.plot([x_envolt[len(x_envolt) - 1], x_envolt[0]], [y_envolt[len(y_envolt) - 1], y_envolt[0]], 'g-')
plt.show()

# Varredura

In [ ]:
# Verifica se o ponto p3 está na semireta p1p2. 
# p1, p2 e p3 são colineares
def on_segment(p1:Dot, p2:Dot, p3:Dot):
    p1HasLessX = p1.x < p2.x 
    if p1HasLessX and p1.x <= p3.x and p2.x >= p3.x:
        return True
    if p2.x <= p3.x and p1.x >= p3.x:
        return True
    return False

In [ ]:
def direction(a, b, c):
    # return 1 if turn left
    # return -1 if turn right
    # return 0 if is co-linear
    
    B = b - a
    C = c - a

    term1 = (B.x)*(C.y)
    term2 = (B.y)*(C.x)
    return (term1 - term2) 

In [ ]:
def aux_segments_intersect(p1, p2, p3, p4):
    d1 = direction(p3, p4, p1)
    d2 = direction(p3, p4, p2)
    d3 = direction(p1, p2, p3)
    d4 = direction(p1, p2, p4)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or  (d3 < 0 and d4 > 0)):
        return True
    if d1 == 0 and on_segment(p3, p4, p1):
        return True
    if d2 == 0 and on_segment(p3, p4, p2):
        return True
    if d3 == 0 and on_segment(p1, p2, p3):
        return True
    if d4 == 0 and on_segment(p1, p2, p4):
        return True
        
    return False

In [ ]:
def segments_intersect(s1:Segment, s2:Segment):
    return aux_segments_intersect(s1.left, s1.right, s2.left, s2.right)

In [ ]:
def isAbove(avl, key):
    if avl.__contains__(key):
        try:
            segment = avl.succ_item(key)[1] 
            return (True, segment)
        except:
            return False, 'error'
    else:
        try:
            segment = avl.ceiling_item(key)[1]
            return (True, segment)
        except:
            return False, 'error'

In [ ]:
def isBelow(avl, key):
    if avl.__contains__(key):
        try:
            segment = avl.prev_item(key)[1] 
            return (True, segment)
        except:
            return False, 'error'
    else:
        try:
            segment = avl.floor_item(key)[1]
            return (True, segment)
        except:
            return False, 'error'
    

In [ ]:
import bintrees

def sweepLineIntersection(endpoitsList, segmentsList):
    avl = [bintrees.AVLTree(), bintrees.AVLTree()]

    for p in endpoitsList:

        s = segmentsList[p.segmentIndx]

        # print(20 * '-')
        # print(avl[0])
        # print(avl[1])
        # print(s, '\t', p.segmentIndx)

        # Insere o segmento na arvore
        if p.endpointType == 'left':

            # Insere o segmento na arvore
            if avl[s.label].__contains__(s.left.y):
                itemList = avl[s.label].get(s.left.y)
                itemList.append(s)
                avl[s.label].insert(s.left.y, itemList)
            else:
                avl[s.label].insert(s.left.y, [s])

            aboveExist, otherSegments = isAbove(avl[s.label - 1], s.left.y)
            if (aboveExist):
                for other in otherSegments:
                    hasIntersection = segments_intersect(s, other)
                    # print("above has Intersect: ", hasIntersection)
                    if (hasIntersection):
                        return True
            
            belowExist, otherSegments = isBelow(avl[s.label - 1], s.left.y)
            if (belowExist):
                for other in otherSegments:
                    hasIntersection = segments_intersect(s, other)
                    # print("below has Intersect: ", hasIntersection)
                    if (hasIntersection):
                        return True
            
            # print("isAbove: ", aboveExist, "\t isBelow: ", belowExist)
            # print()

        
        else:
            aboveExistOther, aboveSegmentsOther = isAbove(avl[s.label - 1], s.right.y)
            belowExistOther, belowSegmentsOther = isBelow(avl[s.label - 1], s.right.y)

            aboveExist, aboveSegments = isAbove(avl[s.label], s.right.y)
            belowExist, belowSegments = isBelow(avl[s.label], s.right.y)

            # print("aboveExist: ", aboveExist)
            # print("belowExist: ", aboveExist)
            # print("aboveExistOther: ", aboveExistOther)
            # print("belowExistOther: ", belowExistOther)


            if (aboveExist and belowExistOther):
                for aboveSegment in aboveSegments:
                    for belowSegment in belowSegmentsOther:
                        hasIntersection = segments_intersect(aboveSegment, belowSegment)    
                        if(hasIntersection):
                            return True


            if (aboveExistOther and belowExist):
                for aboveSegment in aboveSegmentsOther:
                    for belowSegment in belowSegments:
                        hasIntersection = segments_intersect(aboveSegment, belowSegment)    
                        if(hasIntersection):
                            return True

            
            # remoção do segmento
            itemList = avl[s.label].get(s.left.y)
            if len(itemList) > 1:
                 for i in range(len(itemList) - 1):
                    if(itemList[i] != s):
                        continue
                    itemList.pop(i)

            else:
                avl[s.label].pop(s.left.y, False)

    return False



# Teste

Envoltoria 1

In [ ]:
def EnvoltoriaAleatoria(seed=12, numDots=20, x_inicial=0, x_final=100, y_inicial=0, y_final=100):
    dot_list = []

    random.seed(seed)
    for i in range(numDots):
        x = randint(x_inicial, x_final)
        y = noise(randint(y_inicial,y_final))
        a = Dot(x, y)
        dot_list.append(a)

    sorted_list = sortDotsByPolarAngle(dot_list)

    return  (dot_list, Graham(sorted_list))

def plotEnvoltoria(ax, pontos, envoltoria, dotType='r*', envType='b-'):
    
    for i in range(len(pontos)):
        ax.plot(pontos[i].x, pontos[i].y, dotType)
    
    lista_Envoltoria = [[], []]

    for e in envoltoria:
        lista_Envoltoria[0].append(e.x)
        lista_Envoltoria[1].append(e.y)
    
    ax.plot(lista_Envoltoria[0], lista_Envoltoria[1], envType)
    ax.plot([lista_Envoltoria[0][len(lista_Envoltoria[0]) - 1], lista_Envoltoria[0][0]], [lista_Envoltoria[1][len(lista_Envoltoria[1]) - 1], lista_Envoltoria[1][0]], envType)
 

In [ ]:
def preProcessConvexHull(EnvoltoriaA, EnvoltoriaB):
    
    endpoitsList:Endpoint = []
    segmentsList:Segment = []

    for i in range(len(EnvoltoriaA) - 1):
        dotA = EnvoltoriaA[i]
        dotB = EnvoltoriaA[i + 1]

        left = dotA if dotA.x < dotB.x else dotB
        right = dotA if dotA.x >= dotB.x else dotB

        endpoitsList.append(Endpoint(left, segmentIndx=i, endpointType='left'))
        endpoitsList.append(Endpoint(right, segmentIndx=i, endpointType='right'))

        segmentsList.append(Segment(left, right, 0))

    pn = EnvoltoriaA[-1]
    p0 = EnvoltoriaA[0]

    
    left = pn if pn.x < p0.x else p0
    right = pn if pn.x >= p0.x else p0

    segmentMax = len(segmentsList)

    endpoitsList.append(Endpoint(left, segmentIndx=segmentMax, endpointType='left'))
    endpoitsList.append(Endpoint(right, segmentIndx=segmentMax, endpointType='right'))
    segmentsList.append(Segment(left, right, 0))


    for j in range(len(EnvoltoriaA), len(EnvoltoriaA) + len(EnvoltoriaB) - 1):
        i = j - len(EnvoltoriaA)
        dotA = EnvoltoriaB[i]
        dotB = EnvoltoriaB[i + 1]

        left = dotA if dotA.x < dotB.x else dotB
        right = dotA if dotA.x >= dotB.x else dotB

        endpoitsList.append(Endpoint(left, segmentIndx=j, endpointType='left'))
        endpoitsList.append(Endpoint(right, segmentIndx=j, endpointType='right'))

        segmentsList.append(Segment(left, right, 1))
    

    pn = EnvoltoriaB[-1]
    p0 = EnvoltoriaB[0]
    left = pn if pn.x < p0.x else p0
    right = pn if pn.x >= p0.x else p0

    segmentMax = len(segmentsList)

    endpoitsList.append(Endpoint(left, segmentIndx=segmentMax, endpointType='left'))
    endpoitsList.append(Endpoint(right, segmentIndx=segmentMax, endpointType='right'))
    segmentsList.append(Segment(left, right, 1))

    endpoitsList.sort()
    
    return (endpoitsList, segmentsList)


In [ ]:
def testTo(n:int):
    nPoints = 6
    fig, ax = plt.subplots(n)
    fig.set_figheight(2*n)
    plt.subplots_adjust(hspace=0.5)

    for i in range(n):
        (pontosA, EnvoltoriaA) = EnvoltoriaAleatoria(i+1, numDots=nPoints)
        (pontosB, EnvoltoriaB) = EnvoltoriaAleatoria(i+3, nPoints, 30, 200, 50, 200)

        plotEnvoltoria(ax[i], pontosA, EnvoltoriaA)
        plotEnvoltoria(ax[i], pontosB, EnvoltoriaB, dotType='go', envType='y-')

        endPointList, segmentsList = preProcessConvexHull(EnvoltoriaA= EnvoltoriaA, EnvoltoriaB= EnvoltoriaB)

        isIntercect = (sweepLineIntersection(endPointList, segmentsList))
        ax[i].set_title(isIntercect)

    plt.show()

# testTo(15)

# Integração com BD

In [ ]:
# %pip install pandas

# Banana BD

In [ ]:
import pandas as pd
def plotClass(List, ax, dotType='r*', envType='b-', compareCol1=1, compareCol2=2, withNoise=True):
    # Recebe uma tabela com duas colunas referentes ao x e y da classe

    x = []
    y  = []
    dot_list = []
    for row in List.itertuples():
        a = noise(row[compareCol1]) if withNoise else row[compareCol1]
        b = noise(row[compareCol2]) if withNoise else row[compareCol2]
        dot = Dot(a, b)
        x.append(a)
        y.append(b)
        dot_list.append(dot)

    sorted_list = sortDotsByPolarAngle(dot_list)
    env = Graham(sorted_list)
    plotEnvoltoria(ax, dot_list, env, dotType=dotType, envType=envType)

In [ ]:
import pandas as pd

banana = pd.read_csv('./data/banana.csv', delimiter=',')

aType = banana[banana['Class'] == 1]
bType = banana[banana['Class'] == -1]

In [ ]:
fig, ax = plt.subplots()
plotClass(aType, ax)
plotClass(bType, ax, dotType='y.', envType='g-')


# Iris BD

In [ ]:
import pandas as pd

iris = pd.read_csv('./data/iris.csv', delimiter=',')

aType = iris[iris['Class'] == 'Iris-setosa']
bType = iris[iris['Class'] == 'Iris-versicolor']
cType = iris[iris['Class'] == 'Iris-virginica']

fig, ax = plt.subplots()

plotClass(aType, ax)
plotClass(bType, ax, dotType='y.', envType='g-')
plotClass(cType, ax, dotType='b^', envType='r-')

# wine quality

In [ ]:
wine = pd.read_csv('./data/winequality-white.csv', delimiter=',')

aType = wine[wine['Quality'] == 4]
bType = wine[wine['Quality'] == 8]

fig, ax = plt.subplots()

col = (aType.shape)[1]
col2 = (bType.shape)[1]

# for i in range(col):
#     for j in range(col2):

#         fig, ax = plt.subplots()
#         plotClass(aType, ax, dotType='rx', envType='r-', compareCol1=i, compareCol2=j)
#         plotClass(bType, ax, dotType='g.', envType='g-', compareCol1=i, compareCol2=j)
#         plt.show()

# Thyroid

In [ ]:
thyroid = pd.read_csv('./data/thyroid.csv', delimiter=',')

aType = thyroid[thyroid['Class'] == 3][['TSH', 'T3', 'TT4', 'T4U', 'FTI', 'Class']] # hipo
bType = thyroid[thyroid['Class'] == 2][['TSH', 'T3', 'TT4', 'T4U', 'FTI', 'Class']] # hyper
cType = thyroid[thyroid['Class'] == 1][['TSH', 'T3', 'TT4', 'T4U', 'FTI', 'Class']] # normal

fig, ax = plt.subplots()

a = 1
b = 5
plotClass(bType, ax, dotType='yx', envType='g-', compareCol1=a, compareCol2=b, withNoise=False)
plotClass(cType, ax, dotType='r.', envType='b-', compareCol1=a, compareCol2=b, withNoise=False)




# Phoneme


In [ ]:
phoneme = pd.read_csv('./data/phoneme.csv', delimiter=',')

aType = phoneme[phoneme['Class'] == 0]
bType = phoneme[phoneme['Class'] == 1]

fig, ax = plt.subplots()

col = (aType.shape)[1]
col2 = (bType.shape)[1]

for i in range(col):
    for j in range(col2):

        fig, ax = plt.subplots()
        plotClass(aType, ax, dotType='rx', envType='r-', compareCol1=i, compareCol2=j)
        plotClass(bType, ax, dotType='g.', envType='g-', compareCol1=i, compareCol2=j)


# Breast Cancer Wisconsin

In [ ]:
import numpy as np

cancer = pd.read_csv('./data/wdbc.csv', delimiter=',')

aType = cancer[cancer['Class'] == 'M']
bType = cancer[cancer['Class'] == 'B']

col = (aType.shape)[1]

# for i in range(col):
#     for j in range(col):

#         fig, ax = plt.subplots()
#         plotClass(aType, ax, dotType='rx', envType='r-', compareCol1=i, compareCol2=j)
#         plotClass(bType, ax, dotType='g.', envType='g-', compareCol1=i, compareCol2=j)

# plt.show()

# Ionosphere

In [ ]:
ionosphere = pd.read_csv('./data/ionosphere.csv', delimiter=',')

aType = ionosphere[ionosphere['Class'] == 'b']
bType = ionosphere[ionosphere['Class'] == 'g']


# fig, ax = plt.subplots()
# a = 24
# b = 15

# plotClass(aType, ax, dotType='rx', envType='r-', compareCol1=a, compareCol2=b)
# plotClass(bType, ax, dotType='g.', envType='g-', compareCol1=a, compareCol2=b)

# plt.show()

aType[['Pulse24', 'Pulse27']]

Coil2000

In [ ]:
coil2000 = pd.read_csv('./data/coil2000.csv', delimiter=',')

aType = coil2000[coil2000['CARAVAN'] == 0]
bType = coil2000[coil2000['CARAVAN'] == 1]


fig, ax = plt.subplots()

# col = (aType.shape)[1]

# for i in range(col):
#     for j in range(col):

#         fig, ax = plt.subplots()
#         plotClass(aType, ax, dotType='rx', envType='r-', compareCol1=i, compareCol2=j)
#         plotClass(bType, ax, dotType='g.', envType='g-', compareCol1=i, compareCol2=j)
#         plt.show()


movement_libras

In [72]:
movement_libras = pd.read_csv('./data/movement_libras.csv', delimiter=',')

aType = movement_libras[movement_libras['Class'] == 1]
bType = movement_libras[movement_libras['Class'] == 12]

# fig, ax = plt.subplots()

# col = (aType.shape)[1]
# col = (bType.shape)[1]

# for i in range(col):
#     for j in range(col2):

#         fig, ax = plt.subplots()
#         plotClass(aType, ax, dotType='rx', envType='r-', compareCol1=i, compareCol2=j)
#         plotClass(bType, ax, dotType='g.', envType='g-', compareCol1=i, compareCol2=j)
#         print(i, j)
#         plt.show()

0.79691
0.67892
0.72147
0.5648
0.67118
0.83366
0.71567
0.77563
0.77756
0.74468
0.78917
0.83752
0.29014
0.2089
0.19923
0.21857
0.24565
0.25145
0.69246
0.57447
0.4971
0.62282
0.58801
0.61896
